In [1]:
import pandas as pd
import numpy as np
import healpy as hp
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from deepsphere import HealpyGCNN, healpy_layers as hp_layer
import matplotlib.pyplot as plt
import glob
import os
%matplotlib inline
print("GPUs visible to TF:", tf.config.list_physical_devices("GPU"))

2025-06-04 17:47:54.996272: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749052075.009985  143544 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749052075.014724  143544 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749052075.025491  143544 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749052075.025509  143544 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749052075.025510  143544 computation_placer.cc:177] computation placer alr

GPUs visible to TF: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
leaflet_type = 'outer'

all_folders = [
    d for d in os.listdir(".")
    if d.startswith("configf_") and os.path.isdir(d)
]

with open("training_output/outer_test_folders.txt", "r") as f:
    test_folders = [line.strip() for line in f if line.strip()]
with open("training_output/outer_train_folders.txt", "r") as f:
    train_folders = [line.strip() for line in f if line.strip()]

print('all folders', len(all_folders), 'train folders', len(train_folders), 'test folders', len(test_folders))
print(all_folders[0:5], train_folders[:5], test_folders[:5])

all folders 954 train folders 725 test folders 129
['configf_1377', 'configf_1797', 'configf_1157', 'configf_790', 'configf_2396'] ['configf_1966', 'configf_2866', 'configf_3389', 'configf_2407', 'configf_3877'] ['configf_2989', 'configf_577', 'configf_3568', 'configf_2808', 'configf_322']


In [3]:
dummy_dirs = ['configf_4095', 'configf_4092']

In [4]:
import os
import numpy as np
import healpy as hp

# --------------------  USER SETTINGS  --------------------
leaflet_type      = "outer"         # we only handle 'outer' here
lmax_phase_bg     = 44              # <<< SH degree for BACKGROUND (label 0) phase components
lmax_phase_fg     = 22              # <<< SH degree for FOREGROUND (labels 1,2,3) phase components
lmax_radial       = 14              # SH degree for radial map
config_root_glob  = "configf_"      # directories that start with this
analysis_subdir   = "analysis_output"

# File names inside each analysis_output folder
radial_fname      = "area_outer_radial_dist.npy"
phase_files       = [                 # 3 mutually-exclusive phase maps
    "nmfk_phase_map_OUTER_UNS_vs_SAT.npy",
    "nmfk_phase_map_OUTER_SAT_vs_CHOL.npy",
    "nmfk_phase_map_OUTER_CHOL_vs_UNS.npy"
]
N_FOREGROUND_CLASSES_PER_PHASEMAP = 3 # Labels 1, 2, 3
# ----------------------------------------------------------

# -------- helper to shift cluster labels:  NaN→0,  1,2,3 stay positive
def fill_and_shift(arr):
    out           = np.empty_like(arr, dtype=np.float32)
    mask_nan      = np.isnan(arr)
    out[mask_nan] = 0.0             # NaN (true background) becomes 0
    out[~mask_nan] = arr[~mask_nan] + 1.0  # Original 0,1,2 become 1,2,3
    return out
# ----------------------------------------------------------

# Calculate latent dimensions based on new lmax parameters
latent_dim_phase_bg  = hp.Alm.getsize(lmax_phase_bg)
latent_dim_phase_fg  = hp.Alm.getsize(lmax_phase_fg)
latent_dim_radial    = hp.Alm.getsize(lmax_radial)

# -----------------------------------------------------------------
# loop over every configf_* directory
# -----------------------------------------------------------------
all_dirs = sorted([d for d in os.listdir(".")
                   if d.startswith(config_root_glob) and os.path.isdir(d)])

print(f"Found {len(all_dirs)} folders matching '{config_root_glob}*'")

for cfg_idx, cfg in enumerate(all_dirs, 1):
    path = os.path.join(cfg, analysis_subdir)
    try:
        radial_data = np.load(os.path.join(path, radial_fname))
        phase_maps_processed = [fill_and_shift(np.load(os.path.join(path, f)))
                                for f in phase_files]
    except FileNotFoundError as e:
        print(f"[{cfg}] missing expected file – skipped ({e})")
        continue

    n_frames, n_pix = radial_data.shape
    
    # Initialize latent arrays
    # Background: 1 component per phase_map_type (total 3 BG components)
    phase_latent_bg  = np.zeros((n_frames, latent_dim_phase_bg, len(phase_files)), dtype=np.complex128)
    
    # Foreground: N_FOREGROUND_CLASSES_PER_PHASEMAP components per phase_map_type
    num_total_fg_channels = len(phase_files) * N_FOREGROUND_CLASSES_PER_PHASEMAP
    phase_latent_fg  = np.zeros((n_frames, latent_dim_phase_fg, num_total_fg_channels), dtype=np.complex128)
    
    radial_latent = np.zeros((n_frames, latent_dim_radial, 1), dtype=np.complex128)

    for t in range(n_frames):
        # ---------- radial latent (1 channel) ----------
        alm_rad = hp.map2alm(radial_data[t], lmax=lmax_radial)
        radial_latent[t, :, 0] = alm_rad

        # ---------- phase latents with separate lmax for BG and FG ----------
        current_fg_latent_ch_idx = 0 # Index for the combined FG latent array
        for phase_map_type_idx, phase_map_data_for_type in enumerate(phase_maps_processed):
            labels_this_type = phase_map_data_for_type[t] # Values {0,1,2,3}
            
            # Process Background component (label 0) for this phase_map_type
            bg_one_hot_map = (labels_this_type == 0).astype(float)
            alm_bg = hp.map2alm(bg_one_hot_map, lmax=lmax_phase_bg)
            phase_latent_bg[t, :, phase_map_type_idx] = alm_bg
            
            # Process Foreground components (labels 1, 2, 3) for this phase_map_type
            for fg_label_value in range(1, N_FOREGROUND_CLASSES_PER_PHASEMAP + 1): # Iterates 1, 2, 3
                fg_one_hot_map = (labels_this_type == fg_label_value).astype(float)
                alm_fg = hp.map2alm(fg_one_hot_map, lmax=lmax_phase_fg)
                phase_latent_fg[t, :, current_fg_latent_ch_idx] = alm_fg
                current_fg_latent_ch_idx += 1

    # ------------------- save -------------------
    out_phase_bg = os.path.join(cfg, f"{leaflet_type}_sh_phase_latent_bg_lmax{lmax_phase_bg}.npy")
    out_phase_fg = os.path.join(cfg, f"{leaflet_type}_sh_phase_latent_fg_lmax{lmax_phase_fg}.npy")
    out_radial   = os.path.join(cfg, f"{leaflet_type}_sh_radial_latent.npy")
    
    np.save(out_phase_bg, phase_latent_bg)
    np.save(out_phase_fg, phase_latent_fg)
    np.save(out_radial,   radial_latent)

    print(f"[{cfg_idx:>3}/{len(all_dirs)}]  saved BG:{phase_latent_bg.shape} -> {os.path.basename(out_phase_bg)} | "
          f"FG:{phase_latent_fg.shape} -> {os.path.basename(out_phase_fg)} | "
          f"Rad:{radial_latent.shape} -> {os.path.basename(out_radial)}")

print("✅  All folders processed.")

Found 954 folders matching 'configf_*'
[  1/954]  saved BG:(68, 1035, 3) -> outer_sh_phase_latent_bg_lmax44.npy | FG:(68, 276, 9) -> outer_sh_phase_latent_fg_lmax22.npy | Rad:(68, 120, 1) -> outer_sh_radial_latent.npy
[  2/954]  saved BG:(101, 1035, 3) -> outer_sh_phase_latent_bg_lmax44.npy | FG:(101, 276, 9) -> outer_sh_phase_latent_fg_lmax22.npy | Rad:(101, 120, 1) -> outer_sh_radial_latent.npy
[  3/954]  saved BG:(101, 1035, 3) -> outer_sh_phase_latent_bg_lmax44.npy | FG:(101, 276, 9) -> outer_sh_phase_latent_fg_lmax22.npy | Rad:(101, 120, 1) -> outer_sh_radial_latent.npy
[  4/954]  saved BG:(101, 1035, 3) -> outer_sh_phase_latent_bg_lmax44.npy | FG:(101, 276, 9) -> outer_sh_phase_latent_fg_lmax22.npy | Rad:(101, 120, 1) -> outer_sh_radial_latent.npy
[  5/954]  saved BG:(101, 1035, 3) -> outer_sh_phase_latent_bg_lmax44.npy | FG:(101, 276, 9) -> outer_sh_phase_latent_fg_lmax22.npy | Rad:(101, 120, 1) -> outer_sh_radial_latent.npy
[  6/954]  saved BG:(101, 1035, 3) -> outer_sh_phase_l